# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from vaastav.understat import *
from vaastav.getters import *
from config import *
from src.match_names import *

pd.set_option('display.max_columns', 100)

In [3]:
SEASONS = ["2021", "2022", "2023"]

# Getting data

In [4]:
fixtures = get_fixtures_data()
fixtures = pd.read_csv("data/previous/fpl_players.csv")

In [5]:
fixtures

,Unnamed: 0,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts
0,0,2020-21,Aaron Connolly,FWD,Brighton,0,0,-3,0,0.3,78,7,2,0,3.4,1.2,2020-09-14T19:15:00Z,45,5,Chelsea,0,0,0,0,1,0,32205,3.0,1.0,32.0,1,0,0,0,55,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-21,Aaron Cresswell,DEF,West Ham,0,0,11,0,11.2,435,6,2,0,2.2,10.4,2020-09-12T19:00:00Z,90,14,Newcastle,0,0,0,0,1,0,70232,2.0,0.0,0.0,1,0,0,0,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-21,Aaron Mooy,MID,Brighton,0,0,0,0,0.0,60,7,0,0,0.0,0.0,2020-09-14T19:15:00Z,0,5,Chelsea,0,0,0,0,1,0,2010,3.0,1.0,0.0,0,0,0,0,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020-21,Aaron Ramsdale,GK,Sheffield Utd,0,0,12,0,0.0,483,8,2,0,1.8,18.2,2020-09-14T17:00:00Z,90,20,Wolves,0,0,0,0,1,2,462939,2.0,0.0,0.0,1,0,0,0,50,True,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-21,Abdoulaye Doucouré,MID,Everton,0,0,20,1,44.6,512,4,0,0,6.9,20.4,2020-09-13T15:30:00Z,90,17,Spurs,0,0,0,0,1,0,143040,1.0,0.0,4.0,3,0,0,0,55,False,0,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78250,78250,2023-24,Yves Bissouma,MID,Spurs,0,0,17,1,12.9,493,22,0,0,6.2,30.2,2023-08-26T11:30:00Z,73,3,Bournemouth,0,0,0,0,3,0,54809,2.0,0.0,19.0,2,20256,25332,5076,50,False,1,3,2.3,0.12,0.17,0.05,0.54,1.0
78251,78251,2023-24,Zack Steffen,GK,Man City,0,0,0,0,0.0,367,30,0,0,0.0,0.0,2023-08-27T13:00:00Z,0,17,Sheffield Utd,0,0,0,0,3,0,13112,2.0,1.0,0.0,0,-851,185,1036,40,False,0,3,1.5,0.00,0.00,0.00,0.00,0.0
78252,78252,2023-24,Zeki Amdouni,FWD,Burnley,0,0,4,0,0.0,594,25,3,0,0.0,0.0,2023-08-27T13:00:00Z,80,2,Aston Villa,0,0,0,0,3,0,12888,3.0,1.0,0.0,2,-634,1296,1930,55,True,0,3,1.5,0.02,0.06,0.04,2.41,1.0
78253,78253,2023-24,Álvaro Fernández Carreras,DEF,Man Utd,0,0,0,0,0.0,380,28,0,0,0.0,0.0,2023-08-26T14:00:00Z,0,16,Nott'm Forest,0,0,0,0,3,0,28375,2.0,3.0,0.0,0,-1740,1562,3302,40,True,0,3,0.5,0.00,0.00,0.00,0.00,0.0


In [6]:
df_fixtures = fixtures

In [7]:
def team_id_to_name(current_teams, team_id):
    return current_teams[team_id - 1]

In [8]:
# df_fixtures["Squad H"] = df_fixtures["team_h"].apply(lambda x: team_id_to_name(current_teams, x))
# df_fixtures["Squad A"] = df_fixtures["team_a"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["date"] = df_fixtures["kickoff_time"].str[:10]

In [9]:
# epl_data = get_epl_data()
# epl_data = epl_data[0]

In [10]:
epl_seasons = []
for season in SEASONS:
    epl_season = get_epl_data(season)
    epl_season = epl_season[0]
    epl_seasons.append(epl_season)

In [11]:
df = pd.DataFrame(columns=["Team", "xG", "xGA", "Date", "GW"])

for season in epl_seasons:
    for k in season.keys():
        title = team_understat_to_fpl(season[k]['title'])
        for h in season[k]['history']:
            xG = h["xG"]
            xGA = h["xGA"]
            date = h["date"][:10]

            new_row = {
                        "Team": title,
                        "xG": xG,
                        "xGA": xGA,
                        "Date": date
                      }
            df = df.append(new_row, ignore_index=True)

In [12]:
# epl_data

# Basic DF

In [13]:
# df = pd.DataFrame(columns=["Team", "xG", "xGA", "Date", "GW"])

# for k in epl_data.keys():
#     title = team_understat_to_fpl(epl_data[k]['title'])
#     i = 1
#     for h in epl_data[k]['history']:
#         xG = h["xG"]
#         xGA = h["xGA"]
#         date = h["date"][:10]
#         gw = i
        
#         new_row = {
#                     "Team": title,
#                     "xG": xG,
#                     "xGA": xGA,
#                     "Date": date,
#                     "GW": gw
#                   }
#         i += 1
#         df = df.append(new_row, ignore_index=True)

In [14]:
def assign_opponent(x):
    r1 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["opp_team_name"] == x["Team"]) ]
    r2 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["team"] == x["Team"]) ]
    print(r1)
    print(r2)
    if r1.size > 0:
        return r1["team"].unique()
    if r2.size > 0:
        return r2["opp_team_name"].unique()
    return None

In [15]:
for i, row in df.head(5).iterrows():
    df.loc[i, "Opponent"] = assign_opponent(row)

       Unnamed: 0   season                           name position     team  \
24367       24367  2021-22         Ayotomiwa Dele-Bashiru      MID  Watford   
24372       24372  2021-22                    Kwadwo Baah      MID  Watford   
24380       24380  2021-22                    Joshua King      FWD  Watford   
24384       24384  2021-22                     Danny Rose      DEF  Watford   
24395       24395  2021-22                   Stipe Perica      FWD  Watford   
24397       24397  2021-22            Isaac Success Ajayi      FWD  Watford   
24409       24409  2021-22                    Juraj Kucka      MID  Watford   
24426       24426  2021-22          Francisco Femenía Far      DEF  Watford   
24431       24431  2021-22                Emmanuel Dennis      FWD  Watford   
24445       24445  2021-22                  Jeremy Ngakia      DEF  Watford   
24481       24481  2021-22             Nathaniel Chalobah      MID  Watford   
24498       24498  2021-22             Christian Kab

In [16]:
df

,Team,xG,xGA,Date,GW,Opponent
0,Aston Villa,1.137180,1.350360,2021-08-14,NaN,[Watford]
1,Aston Villa,1.184260,0.631630,2021-08-21,NaN,Newcastle
2,Aston Villa,0.431464,1.133120,2021-08-28,NaN,Brentford
3,Aston Villa,1.173490,1.221530,2021-09-11,NaN,Chelsea
4,Aston Villa,0.777716,0.755035,2021-09-18,NaN,Everton
...,...,...,...,...,...,...
1591,Nott'ham Forest,0.960354,3.028030,2023-08-26,NaN,NaN
1592,Nott'ham Forest,1.150500,2.284420,2023-09-02,NaN,NaN
1593,Luton Town,1.885940,4.367480,2023-08-12,NaN,NaN
1594,Luton Town,0.465741,3.414190,2023-08-25,NaN,NaN


# Pivot xG

In [17]:
pivot_xG = pd.pivot_table(df, values='xG', index=['Team'],
                       columns=['GW'], aggfunc=np.sum)
pivot_xG['Summary'] = pivot_xG.sum(axis=1)
pivot_xG.columns = ['GW' + str(col).split(".")[0] for col in pivot_xG.columns[:-1]] + ['Summary']
pivot_xG = pivot_xG.sort_values(by=["Summary"], ascending = False)
pivot_xG.head(20)

,Summary
Team,


# Pivot xGA

In [18]:
pivot_xGA = pd.pivot_table(df, values='xGA', index=['Team'],
                       columns=['GW'], aggfunc=np.sum)
pivot_xGA['Summary'] = pivot_xGA.sum(axis=1)
pivot_xGA.columns = ['GW' + str(col).split(".")[0] for col in pivot_xGA.columns[:-1]] + ['Summary']
pivot_xGA = pivot_xGA.sort_values(by=["Summary"], ascending = False)
pivot_xGA.head(20)

,Summary
Team,


# Summary

In [19]:
summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()
# Add a summary row for each unique 'Team'
# summary_df.append({'Team': 'Summary', 'xG': summary_df['xG'].sum(), 'xGA': summary_df['xGA'].sum()}, ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15008\1851615091.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()


In [20]:
summary_df["Opponent"] = "Summary"

In [21]:
summary_df

,Team,xG,xGA,Opponent
0,Arsenal,146.183315,96.438180,Summary
1,Aston Villa,101.306661,115.456792,Summary
2,Bournemouth,45.574627,77.672974,Summary
3,Brentford,121.512921,105.977318,Summary
4,Brighton,137.510877,105.797566,Summary
5,Burnley,43.713568,67.858620,Summary
6,Chelsea,130.432305,93.766652,Summary
7,Crystal Palace,96.302678,97.673445,Summary
8,Everton,101.067672,137.079189,Summary
9,Fulham,52.280403,82.782101,Summary


# Summary per 90

In [22]:
summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()
summary_df_per_90["Opponent"] = "Summary per 90"

C:\Users\user\AppData\Local\Temp\ipykernel_15008\2074694753.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()


In [23]:
summary_df_per_90

,Team,xG,xGA,Opponent
0,Arsenal,1.850422,1.220736,Summary per 90
1,Aston Villa,1.266333,1.443210,Summary per 90
2,Bournemouth,1.085110,1.849357,Summary per 90
3,Brentford,1.518912,1.324716,Summary per 90
4,Brighton,1.718886,1.322470,Summary per 90
5,Burnley,1.066185,1.655088,Summary per 90
6,Chelsea,1.630404,1.172083,Summary per 90
7,Crystal Palace,1.203783,1.220918,Summary per 90
8,Everton,1.263346,1.713490,Summary per 90
9,Fulham,1.244771,1.971002,Summary per 90


# Weighting

In [24]:
# assigning weighted
for i, row in df[~df["Date"].isna()].iterrows():
    # print(df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")])
    df.loc[i, "Weighted_xG"] = row["xG"] / summary_df_per_90[(summary_df_per_90["Team"] == row["Opponent"]) & (summary_df_per_90["Opponent"] == "Summary per 90")]["xGA"].item()
    df.loc[i, "Weighted_xGA"] = row["xGA"] / summary_df_per_90[(summary_df_per_90["Team"] == row["Opponent"]) & (summary_df_per_90["Opponent"] == "Summary per 90")]["xG"].item()

ValueError: ('Lengths must match to compare', (25,), (1,))

In [ ]:
df.head(50)

# Summary weighted

In [ ]:
df = df.append(summary_df)
df = df.append(summary_df_per_90)
df = df.reset_index()

In [ ]:
summary_weighted_df = df.groupby('Team', as_index=False)['Weighted_xG', "Weighted_xGA"].mean()

In [ ]:
summary_weighted_df.sort_values(by=["Weighted_xGA"], ascending=False)

In [ ]:
# summary_weighted_df["Weighted_xG"].sum() / 20